In [1]:
import qlib
import pandas as pd
from qlib.config import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
from qlib.utils import flatten_dict
from qlib.tests.data import GetData
import time
import mlflow

### data

In [2]:

provider_uri = "../qlib_data/qlib_data"  # target_dir

if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    GetData().qlib_data(target_dir=provider_uri, region=REG_CN)
qlib.init(provider_uri=provider_uri, region=REG_CN)

[10372:MainThread](2023-02-01 15:33:12,862) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[10372:MainThread](2023-02-01 15:33:14,989) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[10372:MainThread](2023-02-01 15:33:14,997) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[10372:MainThread](2023-02-01 15:33:14,999) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': WindowsPath('D:/yupeishan/python_file/qlib_data/qlib_data')}


In [3]:
market = "all"
benchmark = "SH000905"

In [4]:
train_start = "2021-01-01"
train_end ="2021-07-30"
valid_start = "2021-08-01"
valid_end = "2021-09-30"
test_start = "2021-10-01"
test_end = "2021-12-31"

In [5]:
mlflow.end_run()

In [6]:
if __name__ == "__main__":
    
    task = {
    
    'ds_config' : {'class': 'TSDatasetH',
                'module_path': 'qlib.data.dataset',
                'kwargs': {
                    'handler': {
                        'class': 'Alpha158',
                        'module_path': 'qlib.contrib.data.handler',
                        'kwargs': {'start_time': train_start,
                                    'end_time': test_end,
                                    'fit_start_time': train_start,
                                    'fit_end_time': train_end,
                                    'instruments': market,
                                    
                                    'infer_processors': [
                                        {'class': 'FilterCol',
                                        'kwargs': {
                                            'fields_group': 'feature',
                                            'col_list': ["RESI5", "WVMA5", "RSQR5", "KLEN", "RSQR10", "CORR5", "CORD5", "CORR10", 
                            "ROC60", "RESI10", "VSTD5", "RSQR60", "CORR60", "WVMA60", "STD5", 
                            "RSQR20", "CORD60", "CORD10", "CORR20", "KLOW"]

                                        }},
                                        {'class': 'RobustZScoreNorm',
                                        'kwargs': {'fields_group': 'feature', 'clip_outlier': True}},
                                        {'class': 'Fillna', 'kwargs': {'fields_group': 'feature'}}],
                                    'learn_processors': [{'class': 'DropnaLabel'},
                                                        # 对预测的目标进行截面排序处理
                                                        {'class': 'CSRankNorm', 'kwargs': {'fields_group': 'label'}}
                                                        
                                                       ],
                                    
                                    # 预测的目标
                                    'label': ['Ref($vwap, -2) / Ref($vwap, -1) - 1']}},
                    
                    'segments': {'train': [train_start, train_end],
                                'valid': [valid_start, valid_end],
                                'test': [test_start, test_end]},
                    'step_len': 20}},
    
    'model_config': {'class': 'GRU',
                    'module_path': 'qlib.contrib.model.pytorch_gru_ts_chg_daybyday',
                    'kwargs': {
                        'd_feat': 20,
                        'hidden_size': 64,
                        'num_layers': 2,
                        'dropout': 0.0,
                        'n_epochs': 200,
                        'lr': 0.001,
                        'early_stop': 10,
                        'batch_size': 800,
                        'metric': 'loss',
                        'loss': 'batch_ic',
                        'n_jobs': 1,
                        'GPU': 0,
                        'seed': None,
                    }}
    }
    
    # 实例化数据集及模型
    t_start = time.time()

    print("初始化ds")
    ds = init_instance_by_config(task["ds_config"])
    t_end = time.time()
    print('ds耗时：%.3fs'%(t_end-t_start))
    t_start = time.time()

    
    print("初始化md")
    
    
    model = init_instance_by_config(task["model_config"])
    t_end = time.time()
    print('md耗时：%.3fs'%(t_end-t_start))
    print("train")
    print("开始训练")
    t_start = time.time()

    # start exp to train model
    with R.start(experiment_name="train_model"):
        R.log_params(**flatten_dict(task))
        model.fit(dataset=ds)
        t_end = time.time()
        print('训练耗时：%.3fs'%(t_end-t_start))
        R.save_objects(trained_model=model)
        rid = R.get_recorder().id

        print("训练结束")
        
        # prediction
        recorder = R.get_recorder()
        ba_rid = recorder.id
        sr = SignalRecord(model, ds, recorder)
        sr.generate()
    
   
   
    



初始化ds


c:\Users\lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\qlib\utils\__init__.py:757: FutureWarning: MultiIndex.is_lexsorted is deprecated as a public function, users should use MultiIndex.is_monotonic_increasing instead.
  if idx.is_monotonic_increasing and not (isinstance(idx, pd.MultiIndex) and not idx.is_lexsorted()):
[10372:MainThread](2023-02-01 15:47:25,523) INFO - qlib.timer - [log.py:113] - Time cost: 850.059s | Loading data Done
[10372:MainThread](2023-02-01 15:47:38,019) INFO - qlib.timer - [log.py:113] - Time cost: 0.281s | FilterCol Done
c:\Users\lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
c:\Users\lenovo\A

ds耗时：868.160s
初始化md
Please install necessary libs for CatBoostModel.


[10372:MainThread](2023-02-01 15:47:49,926) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:61] - GRU pytorch version...
[10372:MainThread](2023-02-01 15:47:49,937) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:79] - GRU parameters setting:
d_feat : 20
hidden_size : 64
num_layers : 2
dropout : 0.0
n_epochs : 200
lr : 0.001
metric : loss
batch_size : 800
early_stop : 10
optimizer : adam
loss_type : batch_ic
device : cpu
n_jobs : 1
use_GPU : False
seed : None
[10372:MainThread](2023-02-01 15:47:50,035) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:124] - model:
GRUModel(
  (rnn): GRU(20, 64, num_layers=2, batch_first=True)
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
)
[10372:MainThread](2023-02-01 15:47:50,037) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:125] - model size: 0.0396 MB
[10372:MainThread](2023-02-01 15:47:50,043) INFO - qlib.workflow - [expm.py:270] - No tracking URI is provided. Use the default tracking URI.
[10372:MainThread](2023-02-01 15:47:50,047) 

md耗时：6.425s
train
开始训练


...

day by day!!!!!
[4405, 4403, 4402, 4399, 4396, 4393, 4388, 4387, 4383, 4383, 4379, 4378, 4377, 4377, 4377, 4376, 4375, 4374, 4373, 4367, 4363, 4359, 4357, 4357, 4351, 4351, 4347, 4339, 4338, 4334, 4329, 4328, 4327, 4326, 4324, 4323, 4323, 4319, 4318, 4317, 4316, 4313, 4311, 4309, 4308, 4306, 4303, 4303, 4297, 4292, 4288, 4288, 4279, 4276, 4274, 4272, 4272, 4269, 4265, 4262, 4259, 4258, 4257, 4250, 4249, 4245, 4243, 4241, 4240, 4239, 4236, 4236, 4234, 4234, 4232, 4230, 4228, 4226, 4222, 4222, 4219, 4216, 4213, 4212, 4212, 4211, 4208, 4206, 4202, 4200, 4198, 4188, 4188, 4188, 4187, 4185, 4183, 4182, 4182, 4181, 4181, 4178, 4178, 4177, 4176, 4176, 4172, 4171, 4171, 4171, 4171, 4169, 4167, 4167, 4164, 4157, 4153, 4153, 4152, 4152, 4151, 4146, 4145, 4140, 4137, 4135, 4132, 4131, 4127, 4127, 4125, 4125, 4124, 4124, 4123, 4123, 4122, 4121, 4118, 4118]
[4479, 4478, 4475, 4471, 4470, 4469, 4467, 4467, 4467, 4466, 4466, 4466, 4465, 4461, 4460, 4455, 4452, 4449, 4447, 4447, 4447, 4447, 4447, 4446

[10372:MainThread](2023-02-01 15:48:06,309) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:269] - training...
[10372:MainThread](2023-02-01 15:48:06,310) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:273] - Epoch0:
[10372:MainThread](2023-02-01 15:48:06,310) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:274] - training...
[10372:MainThread](2023-02-01 15:54:20,315) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:276] - evaluating...
[10372:MainThread](2023-02-01 15:59:35,043) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:279] - train 0.056602, valid 0.080137
[10372:MainThread](2023-02-01 15:59:35,136) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:273] - Epoch1:
[10372:MainThread](2023-02-01 15:59:35,140) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:274] - training...
[10372:MainThread](2023-02-01 16:06:20,165) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:276] - evaluating...
[10372:MainThread](2023-02-01 16:11:43,931) INFO - qlib.GRU - [pytorch_gru_ts_chg_draft.py:279] - train 0

训练耗时：9548.938s
训练结束


c:\Users\lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\qlib\utils\__init__.py:757: FutureWarning: MultiIndex.is_lexsorted is deprecated as a public function, users should use MultiIndex.is_monotonic_increasing instead.
  if idx.is_monotonic_increasing and not (isinstance(idx, pd.MultiIndex) and not idx.is_lexsorted()):
[10372:MainThread](2023-02-01 18:27:56,479) INFO - qlib.workflow - [record_temp.py:159] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 492208354114910911


'The following are prediction results of the GRU model.'
                          score
datetime   instrument          
2021-10-08 SH600000    0.469601
           SH600001    0.295593
           SH600002    0.295593
           SH600003    0.295593
           SH600004   -0.119936


In [7]:
recorder = R.get_recorder(recorder_id=ba_rid, experiment_name="train_model")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
label_df = recorder.load_object("label.pkl")


{'class': 'Recorder', 'id': 'c264c486378948e287e58b8980b7e86b', 'name': 'mlflow_recorder', 'experiment_id': '492208354114910911', 'start_time': '2023-02-01 15:47:50', 'end_time': '2023-02-01 18:28:07', 'status': 'FINISHED'}


c:\Users\lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\qlib\workflow\recorder.py:332: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = self.client.download_artifacts(self.id, name)


In [8]:
label_df.columns = ['label']
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
pred_label



label     score
datetime   instrument                    
2021-10-08 SH600000   -0.003805  0.469601
           SH600001         NaN  0.295593
           SH600002         NaN  0.295593
           SH600003         NaN  0.295593
           SH600004   -0.000132 -0.119936
...                         ...       ...
2021-12-31 SZ301268         NaN  0.295593
           SZ301279         NaN  0.295593
           SZ301288         NaN  0.295593
           SZ301298         NaN  0.295593
           SZT00018         NaN  0.295593

[298412 rows x 2 columns]

In [9]:
pred_label = pred_label.dropna(axis=0,how='any')
pred_label

label     score
datetime   instrument                    
2021-10-08 SH600000   -0.003805  0.469601
           SH600004   -0.000132 -0.119936
           SH600006   -0.002019  0.098792
           SH600007   -0.019457  0.260215
           SH600008   -0.033927 -0.305151
...                         ...       ...
2021-12-31 SZ301198   -0.044113 -0.093017
           SZ301199   -0.032918 -0.235070
           SZ301211   -0.078627 -0.010728
           SZ301213   -0.072030 -0.309071
           SZ301221   -0.065021 -0.627428

[276839 rows x 2 columns]

In [10]:
pred_label.reset_index().groupby('datetime').apply(lambda df: df['label'].corr(df['score'])).reset_index()[0].mean()


0.014108799910146049